In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if a GPU is available
print(torch.cuda.device_count())  # Should return the number of GPUs
print(torch.cuda.get_device_name(0))  # Should return the name of your GPU (e.g., GTX 1070)

In [ ]:
# Prepare the directory for a single transformation
import os
import random
import shutil

# Define the base directories
base_dir = 'celeba_Orig'
images_dir = os.path.join(base_dir, 'images')
output_dir = os.path.join(base_dir, 'reduced/images')
# Create new directory structure
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
val_dir = os.path.join(output_dir, 'val')

# Create the directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

#Behavior of the Dataset and Loader ==> Preprocessing (preprocess Method): The preprocess method divides the dataset into two parts:
# The first 2000 entries go into self.test_dataset. The rest go into self.train_dataset.
num_test_images = 2000 #int(len(image_files) * 0.10)
test_images = random.sample(image_files, num_test_images)

# Move the images to the test folder
for image in test_images:
    shutil.copy(os.path.join(images_dir, image), os.path.join(test_dir, image))

In [ ]:
# Prepare list_attr_celeba.txt for the single transformation
import os
import pandas as pd

# Define the path to the file
file_path = "celeba_orig/list_attr_celeba.txt"  # Replace with the actual file path

# Read the file starting from the second line, and add 'image_id' as the first column
column_names = ['image_id'] + [
    "5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs",
    "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows",
    "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones",
    "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin",
    "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair",
    "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace",
    "Wearing_Necktie", "Young"
]

# Read the file, skipping the first line and adding column names
df = pd.read_csv(file_path, sep=r'\s+', header=None, skiprows=2, names=column_names)

# Define file paths
reduced_images_dir = "celeba_orig/reduced/images/test"
reduced_attr_file = "celeba_orig/reduced/list_attr_celeba.txt"

# Step 1: Get the list of image files in the reduced directory
reduced_images = set(os.listdir(reduced_images_dir))  # Get filenames in the directory

# Step 2: Filter the dataframe for rows where the image ID is in the reduced images
df_filtered = df[df['image_id'].isin(reduced_images)]

# Step 3: Open the file in write mode
with open(reduced_attr_file, 'w') as f:
    # Write the number of rows in df_filtered
    f.write(f"{len(df_filtered)}\n")
    
    # Write the column names excluding 'image_id' (join the rest with spaces)
    f.write(" ".join(df_filtered.columns[1:]) + "\n")
    
    # Write the data in df_filtered (row by row) to avoid extra new lines
    for index, row in df_filtered.iterrows():
        # Write each row to the file, with space-separated values
        f.write(" ".join(row.astype(str).values[0:]) + "\n")

In [ ]:
# single-transformation
# Mode Selection in __getitem__: If the mode is 'test', it fetches items from self.test_dataset.
# stargan code was copied to new directory and solver was adjusted so it save images seperately
!python ./stargan_code/main.py --mode test --dataset CelebA --image_size 128 --c_dim 5 \
                 --selected_attrs Black_Hair Blond_Hair Brown_Hair Male Young \
                 --model_save_dir="." \
                 --result_dir='stargan_celeba_128/results' \
                 --celeba_image_dir="./celeba_Orig/reduced/images/test" \
                 --attr_path="./celeba_Orig/reduced/list_attr_celeba.txt"

In [ ]:
# Post-process the output for single transformation
import os
import random
import shutil

#single transformation: Black to blond hair
    # Define source and destination directories
source_dir = r"stargan_celeba_128/results"  # Source directory
destination_dir = r"celeba_Orig/reduced/images/single transformation/images" 
os.makedirs(destination_dir, exist_ok=True)

    # copy files with blond hair to single transformation directory
for filename in os.listdir(source_dir):
    if filename.endswith("Blond_Hair.jpg"):  # Check if file ends with 'Blond_Hair.jpg'
        src_path = os.path.join(source_dir, filename)  # Full path to the source file
        dest_path = os.path.join(destination_dir, filename)  # Full path to the destination
        shutil.copy(src_path, dest_path)  # Copy file

    # delete files in the source directory (so new run of model can take place)
for filename in os.listdir(source_dir):
    item_path = os.path.join(source_dir, filename)  # Full path of the item
    # Check if it is a file or a directory
    if os.path.isfile(item_path) or os.path.islink(item_path):
            os.remove(item_path)  # Remove the file or symlink

In [ ]:
# Prepare the single transformation directory for a further transformation
import os
import random
import shutil

# Define the base directories
base_dir = 'celeba_Orig'
images_dir = os.path.join(base_dir, 'reduced/images/single transformation/images')
output_dir = os.path.join(base_dir, 'reduced/images/single transformation/')
# Create new directory structure
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
val_dir = os.path.join(output_dir, 'val')

# Create the directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

#Behavior of the Dataset and Loader ==> Preprocessing (preprocess Method): The preprocess method divides the dataset into two parts:
# The first 2000 entries go into self.test_dataset. The rest go into self.train_dataset.
num_test_images = int(len(image_files))
test_images = random.sample(image_files, num_test_images)

# Move the images to the test folder
for image in test_images:
    source_path = os.path.join(images_dir, image)
    
    # Extract the new file name (part before the underscore + .jpg)
    new_name = image.split('_')[0] + '.jpg'
    
    # Destination file path with the new name
    destination_path = os.path.join(test_dir, new_name)
    
    # Copy the file to the new location with the new name
    shutil.copy(source_path, destination_path)

In [ ]:
# Prepare list_attr_celeba.txt for a further transformation
import os
import pandas as pd

# Define the path to the file
file_path = "celeba_orig/list_attr_celeba.txt"  # Replace with the actual file path

# Read the file starting from the second line, and add 'image_id' as the first column
column_names = ['image_id'] + [
    "5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs",
    "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows",
    "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones",
    "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin",
    "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair",
    "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace",
    "Wearing_Necktie", "Young"
]

# Read the file, skipping the first line and adding column names
df = pd.read_csv(file_path, sep=r'\s+', header=None, skiprows=2, names=column_names)

# Define file paths
reduced_images_dir = "celeba_orig/reduced/images/single transformation/test"
reduced_attr_file = "celeba_orig/reduced/images/single transformation/list_attr_celeba.txt"

# Step 1: Get the list of image files in the reduced directory
reduced_images = set(os.listdir(reduced_images_dir))  

# Step 2: Filter the dataframe for rows where the image ID is in the reduced images
df_filtered = df[df['image_id'].isin(reduced_images)]

# Step 3: Set the attributes correct - the model switches (see def create_labels in solvers.py)
df_filtered['Young'] = df_filtered['Young'].apply(lambda x: -1 if x == 1 else x)
df_filtered['Blond_Hair'] = 1
df_filtered['Black_Hair'] = -1
df_filtered['Brown_Hair'] = -1

# Step 4: Open the file in write mode
with open(reduced_attr_file, 'w') as f:
    # Write the number of rows in df_filtered
    f.write(f"{len(df_filtered)}\n")
    
    # Write the column names excluding 'image_id' (join the rest with spaces)
    f.write(" ".join(df_filtered.columns[1:]) + "\n")
    
    # Write the data in df_filtered (row by row) to avoid extra new lines
    for index, row in df_filtered.iterrows():
        # Write each row to the file, with space-separated values
        f.write(" ".join(row.astype(str).values[0:]) + "\n")

In [ ]:
# multi-transformation: the transformed blond hairs to young 
# stargan code was copied to new directory and get_loader was adjusted so it doesn't perform another crop/resize
# Mode Selection in __getitem__: If the mode is 'test', it fetches items from self.test_dataset.
!python ./stargan_code2/main.py --mode test --dataset CelebA --image_size 128 --c_dim 5 \
                 --selected_attrs Black_Hair Blond_Hair Brown_Hair Male Young \
                 --model_save_dir="." \
                 --result_dir='stargan_celeba_128/results' \
                 --celeba_image_dir="./celeba_Orig/reduced/images/single transformation/test" \
                 --attr_path="./celeba_Orig/reduced/images/single transformation/list_attr_celeba.txt"

In [ ]:
# Post-process the output for multi transformation
import os
import random
import shutil

#further transformation using the same generator: blond hair to young
    # Define source and destination directories
source_dir = r"stargan_celeba_128/results"  # Source directory
destination_dir = r"celeba_Orig/reduced/images/multi transformation/images" 
os.makedirs(destination_dir, exist_ok=True)

    # copy files with blond hair to multi transformation directory
for filename in os.listdir(source_dir):
    if filename.endswith("Young.jpg"):  # Check if file ends with 'Young.jpg'
        src_path = os.path.join(source_dir, filename)  # Full path to the source file
        new_name = filename.split('_')[0] + '.jpg'
        # Destination file path with the new name
        dest_path = os.path.join(destination_dir, new_name)  # Full path to the destination
        shutil.copy(src_path, dest_path)  # Copy file

    # delete files in the source directory (so new run of model can take place)
for filename in os.listdir(source_dir):
    item_path = os.path.join(source_dir, filename)  # Full path of the item
    # Check if it is a file or a directory
    if os.path.isfile(item_path) or os.path.islink(item_path):
            os.remove(item_path)  # Remove the file or symlink